<a href="https://colab.research.google.com/github/JuanBarros2/NLP/blob/master/Agrupando_letras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupando letras de Rock

Letras de músicas segundo o vagalume.com.br, coletadas por Anderson Neisse e [disponíveis no kaggle](https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres/data).

Os dados originais foram tratados para diminuir duplicações e diminuir o tamanho do arquivo. Os resultados (e outros recortes dos dados de letras) estão [nesse repo](https://github.com/nazareno/palavras-nas-letras).


In [3]:
import pandas as pd
import numpy as np
import altair as alt

In [4]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
letras = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-rock-grande.csv')

letras.size

53760

In [26]:
letras['Artist'].value_counts()[0:20]

Lulu Santos                462
Engenheiros do Hawaii      402
Jota Quest                 296
Skank                      290
Rita Lee                   282
Erasmo Carlos              236
Cássia Eller               226
Titãs                      209
Charlie Brown Jr           198
Raul Seixas                196
Capital Inicial            195
Biquini Cavadão            184
Os Paralamas do Sucesso    181
Barão Vermelho             164
Velhas Virgens             156
Fresno                     153
Blitz                      150
Pato Fu                    146
Ira!                       142
Rosa de Saron              137
Name: Artist, dtype: int64

## Pré processamento do texto

In [27]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

204


In [28]:
#construct a new list to store the cleaned text
clean_lyrics = []
for w in range(len(letras.Lyric)):
    lyric = letras['Lyric'][w]

    #remove special characters and digits
    lyric  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",lyric )
    lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_lyrics.append(lyric)

clean_lyrics[0:10]

['Enquanto ela estiver aqui Ainda haverá amor Com ela estou feliz com ela enfrento dor Não adianta fugir não adianta chorar dia ela sumir nada mais irá sobrar Sonhar viver todo dia agradecer rezar pra você ser última morrer Podem tentar atingir Podem mandar pra onde for Enquanto ela estiver aqui vida ainda tem valor Sonhar viver todo dia agradecer rezar pra você ser última morrer Uma gota você vale mais que tudo Quando solução não pode resolver fico com você esperança fico com você Sonhar viver todo dia agradecer rezar pra você ser última morrer Sonhar viver todo dia agradecer rezar pra você ser última morrer esperança Esperança',
 'deu luz fez sonhar Chorou por mim fez amar Foi pra mim Foi tudo mais pouco enfim Canto quando sinto sua falta Rezo pelo bem sua alma queria que você vivesse mais vida levou deixou aqui que você ensinou que faz feliz Que Deus bom lugar guarde pra mim quando for minha vez encontro por Fale mais Quero ouvir Onde está Não vejo mas pra sentir Foi pra mim Foi tud

## Vetores TF-IDF

In [29]:
#TF-IDF vectorizer
tfv = TfidfVectorizer(
        min_df = 10,
        max_df = 0.5,
        max_features = None,
        stop_words = stop_words, 
        ngram_range = (1,3)
  )

#transform
vec_text = tfv.fit_transform(clean_lyrics)

#returns a list of words.
words = tfv.get_feature_names()

len(words)

8525

In [ ]:
words[1:100]

## Agrupamento

### Decidindo K

In [36]:
from sklearn.cluster import MiniBatchKMeans

# escolhendo k
qualidade = pd.DataFrame(columns = ['k', 'ssd'])
for k in range(1,170, 1):
  # kmeans = KMeans(n_clusters=k, random_state=0)
  kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
  kmeans.fit(vec_text)
  qualidade = qualidade.append({'k': k, 'ssd' : kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

alt.Chart(...)

### Agrupando com o K escolhido

In [39]:
kmeans = MiniBatchKMeans(n_clusters=17, init_size=1024, batch_size=2048, random_state=20)
#fit the data 
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

letras_ag = letras.assign(grupo = labels)

letras_ag.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,grupo
2516,O Tudo Que É Nada,"Me deixa andar com os pés no chão. Me deixa ser de novo eu. Me desprendi desse corpo,. Que já ne...",Isabella Taviani,97,1.3,Rock,MPB; Rock; Romântico; Pop/Rock; Pop; Axé; Clássico; Forró; Surf Music; Sertanejo,10
6645,São Paulo Meu Amor Minha Menina,Eu vou chegar ao anoitecer. Vou descer em Congonhas pra poder te ver. Você de noite é deslumbran...,Velhas Virgens,137,2.4,Rock,Rock; Blues; Punk Rock; Hard Rock; Rock Alternativo,10
1813,Análise Descontraída,Análise descontraída. Roberto Carlos / Erasmo Carlos. . Morro sem entender. Porque a ciência con...,Erasmo Carlos,225,2.3,Rock,Rock; Jovem Guarda; Romântico; MPB; Soul Music; Psicodelia; Samba; Dance; Classic Rock; Fado; Fo...,8
5571,Chance,"E o medo se vai. Eu busco no horizonte. Os sonhos que deixei pra trás,. Por não saber viver.. E ...",Rosa de Saron,134,8.2,Rock,Gospel/Religioso; Rock; Pop/Rock; Romântico; Rock Alternativo; Axé; Funk; MPB; Blues; Pagode; Ha...,1
5439,Tão,Tão boazinha. Tão certinha. Tão discreta. Tão correta. Tão modesta. Tão honesta. Tão decente. Tã...,Rita Lee,297,7.6,Rock,Rock; Pop/Rock; MPB; Rock Alternativo; Jovem Guarda; Classic Rock; K-Pop/K-Rock; Bossa Nova; Har...,13
1909,Minha Superstar,MINHA SUPERSTAR – ERASMO CARLOS. QUANDO A LUZ SE APAGA. NO PALCO DO MEU QUARTO. E O PERFUME AVIS...,Erasmo Carlos,225,2.3,Rock,Rock; Jovem Guarda; Romântico; MPB; Soul Music; Psicodelia; Samba; Dance; Classic Rock; Fado; Fo...,8
5303,Entre Sem Bater,A vida para mim estava morta. Mas você bateu na minha porta. E me encontrou meio perdida. Uma Li...,Rita Lee,297,7.6,Rock,Rock; Pop/Rock; MPB; Rock Alternativo; Jovem Guarda; Classic Rock; K-Pop/K-Rock; Bossa Nova; Har...,8
6447,E o Meu Peito Mais Aberto Que o Mar da Bahia,Quando eu chego em casa e você não está. Penso em te procurar. Mesmo sabendo que vais voltar. Se...,Vanguart,92,0.8,Rock,Indie; Rock; Folk; Pop/Rock; MPB; Rock Alternativo; Country; Rap,1
2623,Me Adora (Part. Pitty),Tantas decepções eu já vivi. Aquela foi de longe a mais cruel. Um silêncio profundo e declarei:....,Jota Quest,154,13.3,Rock,Pop/Rock; Pop; Rock; Romântico; Funk; Black Music; Trilha Sonora; Soul Music; Hip Hop; Rock Alte...,8
3290,Inestimável,Esta é para você. Que sabe o que é melhor pra mim. Esta é pra te dizer. Que um ideal não tem fim...,Lulu Santos,271,10.0,Rock,Pop/Rock; Rock; Pop; MPB; Dance; Electronica; Romântico; Funk; Black Music; Gospel/Religioso; Ra...,10


## Interpretando os grupos

In [40]:
letras_ag['grupo'].value_counts()


8     2815
1     1214
9      683
6      612
10     591
4      333
13     257
14     167
7      155
15     141
11     132
16     124
2      119
3      103
12      91
0       80
5       63
Name: grupo, dtype: int64

In [41]:
#this loop transforms the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : homem, onde, exército, caras, mulher, nunca, perdidos, legal, tudo, selva, aranha, eis, vai, relva
1 : tudo, vai, sempre, ser, mim, nada, vida, aqui, hoje, nunca, dia, mundo, assim, pode
2 : assim, baby, tchu, fiz, tchu tchu, bela, então, chove, tempo, baby baby, coração, pena, macaco, vai
3 : dia, lembro, bom, todo dia, pra ser, todo, dia dia, ainda, aqui, bom bom, ser, bom dia, vida, ser sincero
4 : bem, faz, ninguém, vai, ser, tão, sinto, todo, mundo, sei, quero, tudo, todo mundo, vou
5 : mãe, cruel, disco, agora, mãe mãe, love, mamãe, hey, atividade, show, salve, nome, uns, milhas
6 : quero, sei, dizer, mim, posso, ser, tudo, ver, agora, assim, vou, amo, amor, ficar
7 : vamos, gente, país, tudo, todos, vai, viver, então vamos, vida, dia, vamos viver, então, bom, ser
8 : vai, vida, nada, mundo, ninguém, dia, vem, quer, ser, aqui, agora, sei, tão, faz
9 : vou, vai, quero, sei, mim, onde, aqui, tudo, ser, ver, mundo, deixar, ficar, dia
10 : amor, mim, coração, vida, ser, tudo, vai

Quais são algumas músicas do grupo X?

In [45]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 1')[['SName', 'Lyric', 'Artist']].sample(20)

,SName,Lyric,Artist
7180,A Minha Gratidão é Uma Pessoa,Depois de pensar um pouco. Ela viu que não havia mais motivo e nem razão. E pôde perdoá-lo. É fá...,Jota Quest
7625,No Meio do Mar,"Ouve o som, então, tom Tudo vai durar. Só eu passo, só eu morro,. socorro. No meio do mar. Se ma...",Skank
2215,Diga (Parte 2),Tira a maquiagem pra que eu possa ver. Aquilo que você se esforça pra esconder. Agora somos só n...,Fresno
1267,Chuva Quer Cair,Hoje eu acordei sentindo (uuuu). Hoje eu acordei pensando (uuuu). Nesses dias todos sem você. O ...,Danni Carlos
6948,Ton Ton O Nando,"Eu quero pintar um quadro. Ouvir uma música boa. organizar minhas fotos, meu passado. pra não pe...",Danni Carlos
4993,Gospel,Por que é que o Sol nasceu de novo. E não amanheceu?. Por que é que tanta honestidade. No espaço...,Raul Seixas
6469,Amigo,"Outro dia, um amigo veio me falar. Que tinha conhecido um outro lugar. Que era além da vida, que...",Vanguart
2430,Tanto Quanto Eu,Nos dias da semana. Nos anos que virão. Vão correr notícias de abalar o coração. Tudo é tão calm...,Ira!
3884,Eu Não Sei Mentir Direito,No pais do futebol. eu nunca joguei bem. Poderia ser um sintoma. mas meu jogo de cintura. se man...,O Rappa
2445,All In Your Mind,"Tudo em Sua Mente. Olhe dentro de sua cabeça e procure,. me fale agora e está tudo em sua mente....",Iron Maiden


### Artistas mais presentes em cada grupo


In [46]:
for g in range(0, 7):
  print('\n-----\nGRUPO {}:'.format(g))
  print(letras_ag.query('grupo ==  {}'.format(g))['Artist'].value_counts()[0:10])
  print('-----')


-----
GRUPO 0:
Erasmo Carlos            9
Engenheiros do Hawaii    8
Ira!                     6
Velhas Virgens           5
Blitz                    4
PG                       3
Barão Vermelho           3
Cazuza                   3
Raul Seixas              3
Charlie Brown Jr         3
Name: Artist, dtype: int64
-----

-----
GRUPO 1:
Fresno                   63
Jota Quest               60
Engenheiros do Hawaii    56
Nx Zero                  54
Lulu Santos              54
Biquini Cavadão          48
Capital Inicial          48
Rosa de Saron            47
Dead Fish                40
CPM 22                   39
Name: Artist, dtype: int64
-----

-----
GRUPO 2:
Lulu Santos         16
Skank                8
Isabella Taviani     6
Raul Seixas          6
Blitz                6
Capital Inicial      5
Rita Lee             5
Pato Fu              4
Velhas Virgens       4
Leoni                4
Name: Artist, dtype: int64
-----

-----
GRUPO 3:
Engenheiros do Hawaii    10
CPM 22                   10
L

In [19]:
letras_ag.query('Artist ==  "Raça Negra"')[['grupo', 'SName']].groupby('grupo').count()

,SName
grupo,


## Visualizando grupos em muitas dimensões

Para visualizar melhor estruturas de semelhanças nos nossos dados, precisamos que eles estejam em 2D, usaremos um método de **redução de dimensionalidade**.

Os dois método mais frequentemente usados são PCA e t-SNE.

In [20]:
from sklearn.manifold import TSNE

letras_embedded = TSNE(n_components=2, verbose=1).fit_transform(vec_text)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 7680 samples in 0.005s...
[t-SNE] Computed neighbors for 7680 samples in 3.715s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7680
[t-SNE] Computed conditional probabilities for sample 2000 / 7680
[t-SNE] Computed conditional probabilities for sample 3000 / 7680
[t-SNE] Computed conditional probabilities for sample 4000 / 7680
[t-SNE] Computed conditional probabilities for sample 5000 / 7680
[t-SNE] Computed conditional probabilities for sample 6000 / 7680
[t-SNE] Computed conditional probabilities for sample 7000 / 7680
[t-SNE] Computed conditional probabilities for sample 7680 / 7680
[t-SNE] Mean sigma: 0.542049
[t-SNE] KL divergence after 250 iterations with early exaggeration: 123.271011
[t-SNE] KL divergence after 1000 iterations: 3.824572


In [21]:
letras_ag = letras_ag.assign(tsne1 = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras_ag.sample(1000)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'grupo:N',
    tooltip = ['Artist', 'SName']
).interactive()

alt.Chart(...)